In [1]:
from pathlib import Path
import torch
import json
from argparse import Namespace

In [2]:
cd ../code

/media/discoD/repositorios/RE-Context-or-Names/finetune/supervisedRE/code


In [3]:
from model import REModel

In [4]:
cd ../../..

/media/discoD/repositorios/RE-Context-or-Names


In [5]:
from utils.utils import EntityMarker

In [6]:
rel2id_path = Path('finetune/supervisedRE/data/datalawyer/rel2id.json')

In [7]:
rel2id = json.load(rel2id_path.open(mode='r', encoding='utf8'))
rel_num = len(rel2id)
rel_num

5

In [8]:
rel2id

{'no_relation': 0, 'VALOR': 1, 'ATRIBUICAO': 2, 'DECISAO': 3, 'FUNCAO': 4}

In [9]:
args = Namespace()
setattr(args, 'entity_marker', True)
setattr(args, 'hidden_size', 768)
setattr(args, 'rel_num', rel_num)
setattr(args, 'model_name', 'neuralmind/bert-base-portuguese-cased')
setattr(args, 'ckpt_to_load', Path('finetune/supervisedRE/save_dir/datalawyer/model_cp.th'))

In [10]:
model = REModel(args)

********* load from finetune/supervisedRE/save_dir/datalawyer/model_cp.th ***********


In [11]:
entityMarker = EntityMarker(args)

In [12]:
data = {
    "token":["20\u00aa","Vara","do","Trabalho","do","Recife","[unused99]","AVENIDA","MARECHAL","MASCARENHAS","DE","MORAIS",",","4631",",","IMBIRIBEIRA",",","RECIFE","-","PE","-","CEP",":","51150","-","004","[unused99]","RTOrd","0000026","-","86",".","2016",".","5",".","06",".","0020","[unused99]","AUTOR",":","ANTONIO","CARLOS","PINHEIRO"],
    "h":{"name":"ANTONIO CARLOS PINHEIRO","pos":[42,45],"type":"PESSOA"},
    "t":{"name":"AUTOR","pos":[40,41],"type":"FUNCAO"}
}

In [13]:
ids, ph, pt = entityMarker.tokenize(data["token"], data['h']['pos'], data['t']['pos'])
mask = len(ids) * [1]

In [14]:
inputs = {
    "input_ids": torch.tensor([ids], dtype=torch.int64, device=0),
    "mask": torch.tensor([mask], dtype=torch.int64, device=0),
    "h_pos": torch.tensor([ph], dtype=torch.int64, device=0),
    "t_pos": torch.tensor([pt], dtype=torch.int64, device=0),
    "label": None
}

In [15]:
inputs['input_ids'].shape

torch.Size([1, 89])

In [16]:
inputs['mask'].shape

torch.Size([1, 89])

In [17]:
with torch.no_grad():
    model.training = False
    model.eval()
    model.cuda()
    _, output = model(**inputs)

In [18]:
output

tensor([4], device='cuda:0')